In [1]:
!pip install fedot fedot[extra]
!pip uninstall xarray -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - \ done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - done
INFO: pip is looking at multiple versions of salib to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions o

In [2]:
import pandas as pd
from fedot.api.main import Fedot
from fedot.core.pipelines.pipeline_builder import PipelineBuilder

In [3]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
sub = pd.read_csv("/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv")

In [4]:
train.drop(columns=["id", "f_27"], inplace=True)
test.drop(columns=["id", "f_27"], inplace=True)

In [5]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, train_size=0.85, random_state=42)

In [6]:
auto_model = Fedot(
    problem="classification",
    metric=["precision", "accuracy", "roc_auc"],
    preset="best_quality",
    with_tuning=True,
    timeout=5,
    cv_folds=10,
    seed=42,
    n_jobs=4,
    logging_level=10,
    initial_assumption=PipelineBuilder()
    .add_node(
        "lgbm",
    )
    .build(),
    use_pipelines_cache=False,
    use_auto_preprocessing=True
)

2024-07-11 16:44:49,553 - Unknown integration target: 
2024-07-11 16:44:54,190 - Unknown integration target: 
2024-07-11 16:44:58,450 - Unknown integration target: 
2024-07-11 16:45:02,476 - Unknown integration target: 
2024-07-11 16:45:06,571 - Unknown integration target: 
2024-07-11 16:45:15,435 - Unknown integration target: 
2024-07-11 16:45:24,217 - Unknown integration target: 
2024-07-11 16:45:32,787 - Unknown integration target: 
2024-07-11 16:45:33,910 - Unknown integration target: 
2024-07-11 16:45:33,911 - Unknown integration target: 


In [7]:
auto_model.fit(features=train, target="target")

2024-07-11 16:52:26,852 - ApiDataProcessor - Preprocessing data
2024-07-11 16:52:26,855 - ApiDataProcessor - Train Data (Original) Memory Usage: 175.09 MB Data Shapes: ((765000, 30), (765000, 1))
2024-07-11 16:54:14,251 - ApiDataProcessor - Train Data (Processed) Memory Usage: 186.77 MB Data Shape: ((765000, 32), (765000, 1))
2024-07-11 16:54:14,254 - ApiDataProcessor - Data preprocessing runtime = 0:01:47.402100
2024-07-11 16:54:36,849 - AssumptionsHandler - Initial pipeline fitting started
2024-07-11 16:54:38,756 - PipelineNode - Trying to fit pipeline node with operation: lgbm
2024-07-11 16:55:22,455 - PipelineNode - Obtain prediction in pipeline node by operation: lgbm
2024-07-11 16:55:23,994 - AssumptionsHandler - Initial pipeline was fitted successfully
2024-07-11 16:55:23,998 - AssumptionsHandler - Memory consumption for fitting of the initial pipeline in main session: current 486.9 MiB, max: 1941.0 MiB
2024-07-11 16:55:24,002 - ApiComposer - Initial pipeline was fitted in 69.7 

{'depth': 1, 'length': 1, 'nodes': [lgbm]}

In [8]:
prediction = auto_model.predict_proba(features=test)

2024-07-11 17:07:50,185 - PipelineNode - Obtain prediction in pipeline node by operation: lgbm


In [9]:
sub.target = prediction.ravel()
sub.to_csv("submission.csv", index=False)

In [10]:
prediction_valid = auto_model.predict_proba(features=valid)

2024-07-11 17:08:18,955 - PipelineNode - Obtain prediction in pipeline node by operation: lgbm


In [11]:
from sklearn.metrics import roc_auc_score

roc_auc_score(valid.target, prediction_valid)

0.8595260114096817